In [2]:
import simpy

## BASIC1

In [1]:
# https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html

import simpy
SIM_TIME = 20

def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration) #function to create a Timeout event (wait) 

env = simpy.Environment()
env.process(car(env))

env.run(until=SIM_TIME)


Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19


## BASIC2 - classe

In [10]:
# https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html
# unica differenza la classe come contenitore di diverse funzioni e il ciclo run che richiama altre funzioni 

import simpy
SIM_TIME = 20

class Car(object):
    def __init__(self, env):
        self.env = env          # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            yield self.env.process(self.charge(charge_duration))

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

env = simpy.Environment()
car = Car(env)

env.run(until=SIM_TIME)


Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14
Start driving at 19


## BASIC3 - INTERRUZIONE 
#### capire perche solo UNA interruzione all'inizio 

In [5]:
# https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html
# INTERRUZIONE 

import simpy
SIM_TIME = 20

class Car(object):
    def __init__(self, env):
        self.env = env          # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5

            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')            

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()
        
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)


Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


## BASIC4 - Basic Resource Usage 


In [59]:
#https://simpy.readthedocs.io/en/latest/simpy_intro/shared_resources.html

import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)


def car(env, name, bcs, driving_time, charge_duration,carnum):
    yield env.timeout(driving_time)     # Simulate driving to the BCS

    print(env.now,'(+)',('--'*carnum),'%s arriving at %d' % (name, env.now))     # Request one of its charging spots
    with bcs.request() as req:
        yield req
        print(env.now,'(b)',('--'*carnum),'%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print(env.now,'(-)',('--'*carnum),'%s leaving the bcs at %s' % (name, env.now))

for i in range(10):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 7,i))

env.run(until=25)


0 (+)  Car 0 arriving at 0
0 (b)  Car 0 starting to charge at 0
2 (+) -- Car 1 arriving at 2
2 (b) -- Car 1 starting to charge at 2
4 (+) ---- Car 2 arriving at 4
6 (+) ------ Car 3 arriving at 6
7 (-)  Car 0 leaving the bcs at 7
7 (b) ---- Car 2 starting to charge at 7
8 (+) -------- Car 4 arriving at 8
9 (-) -- Car 1 leaving the bcs at 9
9 (b) ------ Car 3 starting to charge at 9
10 (+) ---------- Car 5 arriving at 10
12 (+) ------------ Car 6 arriving at 12
14 (+) -------------- Car 7 arriving at 14
14 (-) ---- Car 2 leaving the bcs at 14
14 (b) -------- Car 4 starting to charge at 14
16 (+) ---------------- Car 8 arriving at 16
16 (-) ------ Car 3 leaving the bcs at 16
16 (b) ---------- Car 5 starting to charge at 16
18 (+) ------------------ Car 9 arriving at 18
21 (-) -------- Car 4 leaving the bcs at 21
21 (b) ------------ Car 6 starting to charge at 21
23 (-) ---------- Car 5 leaving the bcs at 23
23 (b) -------------- Car 7 starting to charge at 23


# Bank Renege . 
- random exit criteria
- Queue visibility
- Sym metrics
### https://simpy.readthedocs.io/en/latest/examples/bank_renege.html

In [147]:
"""
Bank renege example

Covers:

- Resources: Resource
- Condition events

Scenario:
  A counter with a random service time and customers who renege. Based on the program bank08.py from TheBank tutorial of SimPy 2. (KGM)

"""

import random
import simpy

RANDOM_SEED = 42
NEW_CUSTOMERS = 20  # Total number of customers
INTERVAL_CUSTOMERS = 2.0  # Generate new customers roughly every x seconds
CAPACITY = 1
MIN_PATIENCE = 4  # Min. customer patience
MAX_PATIENCE = 8  # Max. customer patience
TOT_RENEGED = 0
TOT_COMPLETED = 0
TOT_CUSTOMER = 0
QUEUE=0
maxQUEUE=0
PRINTALL = 'N'


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Cust%02d' % i, counter,i, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter,j, time_in_bank):
    global TOT_RENEGED, TOT_COMPLETED,TOT_CUSTOMER,QUEUE,maxQUEUE
    """Customer arrives, is served and leaves."""
    arrive = env.now
    if (PRINTALL == 'Y'):
        print(j,('--'*j),'%s: New at %7.4f - Queue=%s' % (name,arrive,QUEUE))
    TOT_CUSTOMER=TOT_CUSTOMER+1
    QUEUE=QUEUE+1
    if (QUEUE>maxQUEUE):
           maxQUEUE=QUEUE

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)
        wait = env.now - arrive

        if req in results:
            # We got to the counter
            if (PRINTALL == 'Y'):
                print(j,('---'*j),'%s: Serving. Waited %6.3f. Clock %7.4f - Queue=%s' % (name, wait,env.now,QUEUE))
            QUEUE=QUEUE-1

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            cycle = env.now - arrive
            if (PRINTALL == 'Y'):
                print(j,('---'*j),'%s: Finished. Cycle %6.3f. Clock %7.4f - Queue=%s' % (name,cycle,env.now,QUEUE))
            TOT_COMPLETED=TOT_COMPLETED+1

        else:
            # We reneged
            if (PRINTALL == 'Y'):
                print(j,('---'*j),'%s: RENEGED after %d. Clock %7.4f - Queue=%s' % (name, wait,env.now,QUEUE))
            TOT_RENEGED = TOT_RENEGED +1 
            QUEUE=QUEUE-1

# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, CAPACITY)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

print('NEW_CUSTOMERS,INTERVAL_CUSTOMERS,CAPACITY,MIN_PATIENCE,MAX_PATIENCE')
print(NEW_CUSTOMERS,INTERVAL_CUSTOMERS,CAPACITY,MIN_PATIENCE,MAX_PATIENCE)
print('PRINTALL:',PRINTALL)
print('TOT_CUSTOMER:',TOT_CUSTOMER)
print('TOT_COMPLETED:',TOT_COMPLETED)
print('TOT_RENEGED:',TOT_RENEGED)
print('%SUCCESS:',TOT_COMPLETED/TOT_CUSTOMER)
print('maxQUEUE:',maxQUEUE)


Bank renege
NEW_CUSTOMERS,INTERVAL_CUSTOMERS,CAPACITY,MIN_PATIENCE,MAX_PATIENCE
20 2.0 1 4 8
PRINTALL: N
TOT_CUSTOMER: 20
TOT_COMPLETED: 8
TOT_RENEGED: 12
%SUCCESS: 0.4
maxQUEUE: 6


# Carwash example

In [58]:
# Carwash example.
# Covers:
# - Waiting for other processes
# - Resources: Resource
# Scenario:
#   A carwash has a limited number of washing machines and defines a washing processes that takes some (random) time.
#   Car processes arrive at the carwash at a random time. If one washing machine is available, they start the washing process and wait for it
#   to finish. If not, they wait until they an use one.

import random
import simpy

RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 3       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes

class Carwash(object):
#     carwash has a limited number of machines (NUM_MACHINES) to clean cars in parallel.  Cars have to request one of the machines. When they got one, they can start the washing processes and wait for it to finish (which     takes `washtime` minutes).
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
#     The washing processes. It takes a ``car`` processes and tries to clean it
        yield self.env.timeout(WASHTIME)
        print("Carwash removed %d%% of %s's dirt." %
              (random.randint(50, 99), car))

def car(env, name, cw):
#     """The car process (each car has a ``name``) arrives at the carwash (``cw``) and requests a cleaning machine.
#     It then starts the washing process, waits for it to finish and leaves to never come back ...
    print('%s arrives at the carwash at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s enters the carwash at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s leaves the carwash at %.2f.' % (name, env.now))

def setup(env, num_machines, washtime, t_inter):
#     Create a carwash, a number of initial cars and keep creating cars approx. every ``t_inter`` minutes."""
    carwash = Carwash(env, num_machines, washtime)   # Create carwash
    for i in range(4):    
        env.process(car(env, 'Car %d' % i, carwash))   # Create 4 initial cars
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Car %d' % i, carwash))      # Create more cars while the simulation is running

# Setup and start the simulation
print('Carwash')
random.seed(RANDOM_SEED)  # This helps reproducing the results

env = simpy.Environment()   # Create an environment 
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))  #start the setup process
env.run(until=SIM_TIME) # Execute!

Carwash
Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Car 4 arrives at the carwash at 1.00.
Car 5 arrives at the carwash at 2.00.
Carwash removed 65% of Car 0's dirt.
Carwash removed 64% of Car 1's dirt.
Car 6 arrives at the carwash at 5.00.
Car 0 leaves the carwash at 5.00.
Car 1 leaves the carwash at 5.00.
Car 2 enters the carwash at 5.00.
Car 3 enters the carwash at 5.00.
Car 7 arrives at the carwash at 7.00.
Car 8 arrives at the carwash at 8.00.
Carwash removed 55% of Car 2's dirt.
Carwash removed 87% of Car 3's dirt.
Car 2 leaves the carwash at 10.00.
Car 3 leaves the carwash at 10.00.
Car 4 enters the carwash at 10.00.
Car 5 enters the carwash at 10.00.
Car 9 arrives at the carwash at 13.00.
Carwash removed 52% of Car 4's dirt.
Carwash removed 51% of Car 5's dirt.
Car 4 leaves the carwash at 15.00.
Car 5 leav